In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier


from sklearn.feature_selection import SelectFromModel 
from sklearn.model_selection import ShuffleSplit

from sklearn.svm import LinearSVC

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import RFE

## Loading Dataset

Please note that before running the code, you will need all the GDSC files. They can be found on the drive (https://drive.google.com/drive/u/1/folders/11omvpOttkdZZgv_ppbtcCbojkuVR-D61) or directly on the GDSC website.

In [2]:
SCORE = "AUC" #how the sensitivity of the drug is computed; other option: "AUC" 
SAVE = False #whether to save the final matrix or not
PATH = "data/Final matrices/" #where your data is located

N_SPLITS = 5
DRUG_NAMES = {"CI-1040": ["MYC_mut","RB1_mut","ERBB2_amp","BRAF_mut","KRAS_mut","NRAS_mut"],
              "PD0325901": ["MYC_mut","RB1_mut","ERBB2_amp","BRAF_mut","KRAS_mut","NRAS_mut"],
              "Refametinib":["MYC_mut","RB1_mut","ERBB2_amp","BRAF_mut","KRAS_mut","NRAS_mut"],
              "VX-11e":["RB1_mut","ERBB2_amp","CCND1_amp","BRAF_mut","KRAS_mut","NRAS_mut"],
              "Afatinib":["KRAS_mut","NRAS_mut","EGFR_amp","ERBB2_amp","FOXP3_del"],
              "Pelitinib":["BRAF_mut","RB1_mut","MAPK1_del","MYC_mut","EGFR_mut","CDKN1B_del"]
             }

### Response

In [3]:
def scale_data(X_train, X_test):
    
    feature_names = list(X_train) # Store the feature names
    
    sc = StandardScaler()  # Defines the scaler
    X_train = pd.DataFrame(sc.fit_transform(X_train))  # Scales the training data
    X_test = pd.DataFrame(sc.transform(X_test))  # Scales the validation data

    # Replace feature names in the database (they are lost during scaling)
    X_train.columns = feature_names
    X_test.columns = feature_names
    
    return X_train, X_test

In [4]:
def lasso_feature_reduction(X_train, X_val, y_train, Cst = 0.01):
    
    clf = LinearSVC(C = Cst, penalty = "l1", dual = False) #SVC(kernel = 'linear', C = Cst)
    clf.fit(X_train.values, y_train)
    
    model = SelectFromModel(clf, prefit=True, threshold=-np.inf, max_features = 300)
    new_X_train = model.transform(X_train.values)
    new_X_val = model.transform(X_val.values)
    
    return pd.DataFrame(new_X_train), pd.DataFrame(new_X_val)


In [5]:
def rf_feature_reduction(X_train, X_val, y_train, N_ESTIMATORS = 500):
    
    clf = RandomForestClassifier(n_estimators = N_ESTIMATORS, n_jobs = -1, class_weight = "balanced", random_state = 32)
    clf = clf.fit(X_train.values, y_train.values)
    
    model = SelectFromModel(clf, prefit=True, threshold=-np.inf, max_features = 300)   
    new_X_train = model.transform(X_train)
    new_X_val = model.transform(X_val)
    
    importances = clf.feature_importances_
    #std = np.std([tree.feature_importances_ for tree in clf.estimators_], axis=0) --> std dev but we don't use it anyway
    
    return pd.DataFrame(new_X_train), pd.DataFrame(new_X_val), importances

In [6]:
def run_PCA(X, NB_COMPONENTS, kf = 5, verbose = 5):

    if verbose > 1:
        print("Running PCA...")
        print("Number of features before PCA: ", len(X.columns))

    # Starting PCA
    my_PCA = PCA(n_components=NB_COMPONENTS)
    reduced_X = pd.DataFrame(my_PCA.fit_transform(np.array(X.values), y=None))
    reduced_X.columns = [f"PC{elem}" for elem in range(NB_COMPONENTS)]
    
    # Plotting
    if verbose > 0:
        fig = plt.figure()
        g = sns.lineplot(x = range(1, NB_COMPONENTS+1), y = my_PCA.explained_variance_ratio_)
        plt.xlabel('Principal Component')
        plt.ylabel('Variance (Component Scores)')
        plt.title('Screen Plot of Principal Components')
        plt.show(); 
    
    if verbose > 1:
        print("Number of combined features after PCA: ",len(list(reduced_X)))
    
    return reduced_X   

In [7]:
def calculate_performances(DRUG_NAME, ML_matrix, PATH, SCORE, N_SPLITS, verbose = 5):
    
    ## Look at the distribution of the response
    
    if verbose > 0:
        print(f"\nDistribution of responses for {DRUG_NAME}")
        fig = plt.figure(figsize = (10,6))
        sns.distplot(ML_matrix[[DRUG_NAME]])
        plt.title("Plot of the distribution of AUC values for "+ DRUG_NAME)
        plt.xlabel("AUC")
        plt.ylabel("Proportion of cell poplations")
        plt.grid(True);
        plt.show()

    #_______________________________________________________________________________________________________#
    
    ## Data Categorisation and feature matrix preparation
    print("\nCategorisation of the data...")
    # The data is divided in quartiles. Upper/lower third quartile thresholds were used for discretization. 
    # Everything that is in the middle is discarded. See [this](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3995541/) paper.
    # Splitting at the median or mean, although keeping more cells, leads to poorer results.
    
    if verbose > 1:
        print("The lower threshold used here is the lower third quartile = ", ML_matrix[DRUG_NAME].quantile([0.25,0.75])[0.25])
        print("The upper threshold used here is the uppder third quartile = ", ML_matrix[DRUG_NAME].quantile([0.25,0.75])[0.75])
    ML_matrix["Response"] = pd.cut(ML_matrix[DRUG_NAME], [np.min(ML_matrix[DRUG_NAME]), ML_matrix[DRUG_NAME].quantile([0.25,0.75])[0.25], ML_matrix[DRUG_NAME].quantile([0.25,0.75])[0.75], np.max(ML_matrix[DRUG_NAME])], labels = ["sensitive","medium", "resistant"])
    ML_matrix = ML_matrix.drop([DRUG_NAME], axis = 1)
    
    #Drop all the "medium" classes and NaNs
    ML_matrix = ML_matrix.drop(ML_matrix[ML_matrix["Response"] == "medium"].index)
    ML_matrix = ML_matrix.dropna()  
    
    # Convert the response to a float
    ML_matrix["Response"] = [0 if x=='resistant' else 1 for x in ML_matrix['Response']]
    
    # Structure of the feature matrix
    print(f"\tStructure of the final matrix for {DRUG_NAME}:")
    print("\t\tNumber of resistant cells: ", len(ML_matrix[ML_matrix["Response"] == 0]))
    print("\t\tNumber of sensitive cells: ", len(ML_matrix[ML_matrix["Response"] == 1]))
    print("\t\tTotal number of cells: ", len(ML_matrix.index))
    print(f'\n\t\tThe baseline accuracy for {DRUG_NAME} is {100*np.round(len(ML_matrix[ML_matrix["Response"] == 1])/len(ML_matrix.index),2)}%')
    
    #_______________________________________________________________________________________________________#
    
    ## Run the different models and feature selection methods
    
    print(f"\nRunning the models with a {N_SPLITS}-fold cross-validation...")
    print("Please wait. This can take up to a minute.")
    
    # Create our feature and response matrix
    X = ML_matrix.drop("Response", axis = 1)
    y = ML_matrix["Response"].astype('float64')
        
    # Define our models    
    models = [SVC(random_state=40),
              LogisticRegression(solver= 'saga', max_iter=5000, random_state=41),
              MLPClassifier(max_iter=5000, random_state=42),
              RandomForestClassifier(n_estimators=300, max_depth = 3, max_leaf_nodes = 10, random_state=43)]
    
    model_names = ['SVM', 'Logistic Regression', 'MLP', 'Random Forest']
    
    # Define our final result matrix
    final_results = pd.DataFrame(columns = model_names)     
          
    # Run all models
    # Run PCA
    final_results = pretty_run(final_results, X, y, models, model_names, N_SPLITS, DRUG_NAME, FS = "PCA", index_name = "Principal Component Analysis", verbose = verbose)  
          
    # Run RFE
    final_results = pretty_run(final_results, X, y, models, model_names, N_SPLITS, DRUG_NAME, FS = "RFE", index_name = "Recursive Feature Elimination", verbose = verbose)
    
    # Run Lasso Feature Selection
    final_results = pretty_run(final_results, X, y, models, model_names, N_SPLITS, DRUG_NAME, FS = "LASSO", index_name = "Lasso Feature Selection", verbose = verbose)
          
    # Run Random Forest Feature Selection
    final_results = pretty_run(final_results, X, y, models, model_names, N_SPLITS, DRUG_NAME, FS = "RF", index_name = "Random Forest Feature Selection", verbose = verbose)
          
    # Run with no feature selection
    final_results = pretty_run(final_results, X, y, models, model_names, N_SPLITS, DRUG_NAME, FS = "None", index_name = "No feature selection", verbose = verbose)
    
    print(f"All the models were run on {DRUG_NAME} successfully!")
    
    return final_results

In [8]:
def pretty_run(final_results, X, y, models, model_names, N_SPLITS, DRUG_NAME, FS, index_name, verbose = 5):
    
    for i in range(len(models)):
        if verbose > 0:
            print('______________________\n')
            print('Running',model_names[i],'and',index_name)
        perf = run_model(X, y, models[i], FS = FS, n_splits = N_SPLITS, verbose = verbose)
        final_results.loc[index_name,model_names[i]] = perf[2]
    
    final_results.loc[index_name,"Drug"] = DRUG_NAME
    return final_results

In [9]:
def run_model(X, y, clf, FS = "None", n_splits = 5, verbose = 5):
    
    train_accuracies, val_accuracies, rocs = [],[],[]
    
    if FS == "PCA":
        X = run_PCA(X, NB_COMPONENTS = len(X.columns), verbose = verbose)
        
    # Define a cross-validation (shuffleSplit here)
    ss = ShuffleSplit(n_splits, test_size=0.2, random_state=0)

    for count, (training_indices, val_indices) in enumerate(ss.split(X, y), 1):
        
        if verbose > 1:
            print(f'Cross-validation: {count}/{n_splits}')

        # Prepare the test and training set     
        X_train = X.iloc[training_indices,:]
        y_train = y.iloc[training_indices]
        X_val = X.iloc[val_indices,:]
        y_val = y.iloc[val_indices]
        if verbose > 1:
            print(f"The validation set corresponds to roughly {np.round(100*(len(X_val.index)/len(X.index)),2)}% of the total data.") 
        
        #Scaling the features -- useful for most classifier except RF and co
        X_train, X_val = scale_data(X_train, X_val)
        
        if FS == "RFE":            
            selector = RFE(estimator = LinearSVC())
            selector = selector.fit(X_train, y_train)
            X_train = X_train[X_train.columns[selector.support_]]
            X_val = X_val[X_val.columns[selector.support_]]
        elif FS == "LASSO":
            X_train, X_val = lasso_feature_reduction(X_train, X_val, y_train)
        elif FS == "RF":
            X_train, X_val, impor = rf_feature_reduction(X_train, X_val, y_train)
        elif (FS == "None") | (FS == "PCA"):
            pass
        else:
            print("Check your feature selection method. Enter either 'RFE', LASSO', 'RF' or 'None'.")
            return
        
        # Fit the classifier
        clf.fit(X_train.values.tolist(), y_train.values)        
        
        # Predict the classes
        y_pred = clf.predict(X_val.values.tolist())
        
        # Calculate the performance metrics 
        train_acc = accuracy_score(y_train.values.tolist(), clf.predict(X_train.values.tolist()))
        val_acc = accuracy_score(y_val.values.tolist(), y_pred)
        #roc_auc = roc_auc_score(y_val.values, clf.predict_proba(X_val.values)[:, 1])
        
        if verbose > 1:
            print(f"Training accuracy {count}: {train_acc}")
            print(f"Validation accuracy {count}: {val_acc}")
            #print(f"ROC AUC {count}: {roc_auc}")
    
        # Add the performances to their corresponding lists
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)
        
        #rocs.append(roc_auc)
    if verbose > 1:
        print("_________________________________________________________________________________")   
    if verbose > 0:
        print(f'\tAverage Training Accuracy: {np.round(100*np.mean(train_accuracies), 2)} +/- {np.round(100*np.std(train_accuracies),2)}%.')
        print(f'\tAverage Validation Accuracy: {np.round(100*np.mean(val_accuracies),2)} +/- {np.round(100*np.std(val_accuracies),2)}%.')
        #print(f'Average AUC {np.round(100*np.mean(np.array(rocs)),2)} +/- {np.round(100*np.std(np.array(rocs)),2)}%.')
    
    models_saved.append([clf, val_acc])
    
    return 100*np.mean(train_accuracies), 100*np.std(train_accuracies), 100*np.mean(val_accuracies), 100*np.std(val_accuracies)#, 100*np.mean(np.array(rocs)), 100*np.std(np.array(rocs))


In [10]:
models_saved = []

def run_all_drugs(DRUG_NAMES, PATH, SCORE, N_SPLITS, verbose = 5):
    
    final_results = pd.DataFrame()
    
    for drug in DRUG_NAMES:

        print(f"Investigating {drug}... ")

        # Read the corresponding ML matrix
        ML_matrix = pd.read_csv(PATH + f"{drug}.csv")
        ML_matrix.set_index("Cell_line", inplace = True)

        print("Number of cells: ", len(ML_matrix.index))
        if verbose > 3:
            ML_matrix.head() #Show the head of the dataframe for that drug

        results_drug = calculate_performances(drug, ML_matrix, PATH, SCORE, N_SPLITS, verbose = verbose)
        
        final_results = pd.concat([final_results,results_drug])
                                          
        print("_________________________________________________________________________________________________________")
        
    final_results.index.name = "Methods"
    
    return final_results.reset_index(drop=False).set_index(["Drug","Methods"])
                                          
final_results = run_all_drugs(DRUG_NAMES, PATH, SCORE, N_SPLITS, verbose = 0)
display(final_results)

Investigating CI-1040... 
Number of cells:  758

Categorisation of the data...
	Structure of the final matrix for CI-1040:
		Number of resistant cells:  190
		Number of sensitive cells:  189
		Total number of cells:  379

		The baseline accuracy for CI-1040 is 50.0%

Running the models with a 5-fold cross-validation...
Please wait. This can take up to a minute.
All the models were run on CI-1040 successfully!
_________________________________________________________________________________________________________
Investigating PD0325901... 
Number of cells:  759

Categorisation of the data...
	Structure of the final matrix for PD0325901:
		Number of resistant cells:  190
		Number of sensitive cells:  189
		Total number of cells:  379

		The baseline accuracy for PD0325901 is 50.0%

Running the models with a 5-fold cross-validation...
Please wait. This can take up to a minute.
All the models were run on PD0325901 successfully!
____________________________________________________________

SVM Logistic Regression  \
Drug        Methods                                                        
CI-1040     Principal Component Analysis     76.3158             76.5789   
            Recursive Feature Elimination    76.8421             76.8421   
            Lasso Feature Selection          76.3158             76.5789   
            Random Forest Feature Selection  76.3158             76.5789   
            No feature selection             76.3158             76.5789   
PD0325901   Principal Component Analysis     72.8947             73.6842   
            Recursive Feature Elimination    72.6316             72.6316   
            Lasso Feature Selection          73.1579             73.6842   
            Random Forest Feature Selection  73.1579             73.6842   
            No feature selection             73.1579             73.6842   
Refametinib Principal Component Analysis     75.8537             76.3415   
            Recursive Feature Elimination    73.4146             73.6585   
            Lasso Feature Selection          75.8537             76.3415   
            Random Forest Feature Selection  75.8537             76.3415   
            No feature selection             75.8537             76.3415   
VX-11e      Principal Component Analysis     69.8795             72.2892   
            Recursive Feature Elimination    70.1205             69.8795   
            Lasso Feature Selection          69.8795             72.2892   
            Random Forest Feature Selection  69.8795             72.2892   
            No feature selection             69.8795             72.2892   
Afatinib    Principal Component Analysis     60.4762             60.7143   
            Recursive Feature Elimination    53.0952             53.0952   
            Lasso Feature Selection          60.4762             60.7143   
            Random Forest Feature Selection  60.4762             60.7143   
            No feature selection             60.4762             60.7143   
Pelitinib   Principal Component Analysis     58.0723             52.7711   
            Recursive Feature Elimination     53.012             52.2892   
            Lasso Feature Selection          58.0723             52.7711   
            Random Forest Feature Selection  58.0723             52.7711   
            No feature selection             58.0723             52.7711   

                                                 MLP Random Forest  
Drug        Methods                                                 
CI-1040     Principal Component Analysis     76.8421       76.3158  
            Recursive Feature Elimination    76.8421       76.3158  
            Lasso Feature Selection          76.5789       76.3158  
            Random Forest Feature Selection  76.5789       76.3158  
            No feature selection             76.5789       76.3158  
PD0325901   Principal Component Analysis     73.4211       72.8947  
            Recursive Feature Elimination    72.6316       72.6316  
            Lasso Feature Selection          73.4211       72.6316  
            Random Forest Feature Selection  73.4211       72.6316  
            No feature selection             73.4211       72.6316  
Refametinib Principal Component Analysis     75.3659       75.3659  
            Recursive Feature Elimination    73.6585       73.4146  
            Lasso Feature Selection          75.3659        74.878  
            Random Forest Feature Selection  75.3659        74.878  
            No feature selection             75.3659        74.878  
VX-11e      Principal Component Analysis     70.8434       70.8434  
            Recursive Feature Elimination    69.8795       69.8795  
            Lasso Feature Selection          71.0843       71.5663  
            Random Forest Feature Selection  71.0843       71.5663  
            No feature selection             71.0843       71.5663  
Afatinib    Principal Component Analysis     60.7143       60.4762  
            Recursive Feature

### Higest Performing Models 

In [19]:
# CI-1040
# Highest performing model: MLP
print('CI-1040 Model:')
ci_1040_model = models_saved[2]
display(ci_1040_model)
    
# PD0325901
# Highest performing model: Logistic Regression
print('PD0325901 Model:')
pd0325901_model = models_saved[21]
display(pd0325901_model)

# Refametinib
# Highest performing model: Logistic Regression
print('Refametinib Model:')
refametinib_model = models_saved[41]
display(refametinib_model)

# VX-11e
# Highest performing model: Logistic Regression
print('VX-11e Model:')
vx_11e_model = models_saved[61]
display(vx_11e_model)

# Afatinib
# Highest performing model: Logistic Regression
print('Afatinib Model:')
afatinib_model = models_saved[81]
display(afatinib_model)

# Pelitinib
# Highest performing model: Random Forest
print('Pelitinib Model:')
pelitinib_model = models_saved[103]
display(pelitinib_model)

CI-1040 Model:


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=5000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=42, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

PD0325901 Model:


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=5000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=41, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

Refametinib Model:


...

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=5000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=41, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

Afatinib Model:


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=5000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=41, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

Pelitinib Model:


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=10,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=43, verbose=0, warm_start=False)

### Final Prediction Function

In [241]:
models = {'CI-1040': ci_1040_model,'PD0325901': \
          pd0325901_model,'Refametinib': refametinib_model,\
          'VX-11e': vx_11e_model,'Afatinib': afatinib_model,\
          'Pelitinib': pelitinib_model}

feature_selection = {"CI-1040": ["MYC_mut","RB1_mut","ERBB2_amp","BRAF_mut","KRAS_mut","NRAS_mut"],
              "PD0325901": ["MYC_mut","RB1_mut","ERBB2_amp","BRAF_mut","KRAS_mut","NRAS_mut"],
              "Refametinib":["MYC_mut","RB1_mut","ERBB2_amp","BRAF_mut","KRAS_mut","NRAS_mut"],
              "VX-11e":["RB1_mut","ERBB2_amp","CCND1_amp","BRAF_mut","KRAS_mut","NRAS_mut"],
              "Afatinib":["KRAS_mut","NRAS_mut","EGFR_amp","ERBB2_amp","FOXP3_del"],
              "Pelitinib":["BRAF_mut","RB1_mut","MAPK1_del","MYC_mut","EGFR_mut","CDKN1B_del"]
             }

import pandas as pd

# Takes in a one-row dataframe with mutation data
def predict(patient_data): 
    
    results = []
    
    # For each drug
    for i in models.keys():
        
        to_predict = patient_data.loc[:,feature_selection.get(i)]
        to_predict = to_predict.fillna(0)
        
        # Predict 
        clf = models.get(i)
        result = clf.predict(to_predict.values)
        probabilities = clf.predict_proba(to_predict.values.tolist())
        results.append([i, result[0], max(probabilities)])
    
    df = pd.DataFrame(data=results, columns = ['Drug', 'Response', 'Probabilities'])
    df['Response'] = ['Resistant' if x==0 else 'Sensitive' for x in df['Response']]
    
    return df

### Sample Prediction

In [245]:
sample_csv = pd.read_csv("sample_csv.csv")

In [246]:
display(sample_csv)

,MYC_mut,RB1_mut,ERBB2_amp,BRAF_mut,NRAS_mut,CCND1_amp,EGFR_amp,FOXP3_del,MAPK1_del,CDKN1B_del,KRAS_mut,ERBB2_amp.1,BRAF_del,NRAS_amp,CCND1_del,EGFR_mut,FOXP3_amp,MAPK1_mut,CDKN1B_amp,KRAS_del
0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0


In [247]:
predict(sample_csv)

,Drug,Response,Probabilities
0,CI-1040,Resistant,"[0.6509975351895083, 0.34900246481049163]"
1,PD0325901,Resistant,"[0.6787835262927988, 0.32121647370720113]"
2,Refametinib,Resistant,"[0.6025512043753293, 0.3974487956246707]"
3,VX-11e,Resistant,"[0.5897279825415633, 0.4102720174584366]"
4,Afatinib,Sensitive,"[0.3160079735449597, 0.6839920264550403]"
5,Pelitinib,Resistant,"[0.5040219765151802, 0.4959780234848199]"
